In [1]:
import pandas as pd
import numpy as np 
import os

In [2]:
path_test_directory = './Data/Test/'
path_tls_csv = os.path.join(path_test_directory, 'tls_test_01.pcapng.csv') 

In [3]:
df_tls_csv = pd.read_csv(path_tls_csv) 
df_tls_csv.head() 

,Unnamed: 0,dst,dst_resolved,dst_oui,dst_oui_resolved,dst_lg,dst_ig,addr,addr_resolved,addr_oui,...,options_sack_dsack_re,options_sack_dsack,handshake_session_ticket_lifetime_hint,handshake_session_ticket_length,handshake_session_ticket,handshake_extensions_session_ticket,resumed,alert_message,analysis_fast_retransmission,padding
0,0,28630a42,f42d06784ee9,f42d06,f42d06784ee9,0,0,c0a80503,f42d06784ee9,f42d06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,b4a397a2,f42d06784ee9,f42d06,f42d06784ee9,0,0,c0a80503,f42d06784ee9,f42d06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,c0a80503,6c2f804a964c,6c2f80,6c2f804a964c,0,0,b4a397a2,6c2f804a964c,6c2f80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,b4a397a2,f42d06784ee9,f42d06,f42d06784ee9,0,0,c0a80503,f42d06784ee9,f42d06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,b4a397a2,f42d06784ee9,f42d06,f42d06784ee9,0,0,c0a80503,f42d06784ee9,f42d06,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
dict_col_value = {}
for col in df_tls_csv.columns: 
    dict_col_value[col] = list(df_tls_csv[col].unique())

Index(['Unnamed: 0', 'srcport', 'dstport', 'port', 'stream', 'completeness',
       'completeness_rst', 'completeness_fin', 'completeness_data',
       'completeness_ack',
       ...
       'reassembled_in', 'options_sack_dsack_le', 'options_sack_dsack_re',
       'options_sack_dsack', 'handshake_session_ticket_lifetime_hint',
       'handshake_session_ticket_length', 'handshake_session_ticket',
       'resumed', 'alert_message', 'analysis_fast_retransmission'],
      dtype='object', length=261)


In [5]:
df_tls_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1983 entries, 0 to 1982
Columns: 261 entries, Unnamed: 0 to analysis_fast_retransmission
dtypes: float64(156), int64(1), object(104)
memory usage: 3.9+ MB


In [6]:
df_tls_csv.describe()

,Unnamed: 0,stream,completeness,completeness_rst,completeness_fin,completeness_data,completeness_ack,completeness_syn_ack,completeness_syn,completeness_str,...,handshake_server_curve_type,handshake_server_point_len,handshake_sig_len,handshake_client_point_len,analysis_spurious_retransmission,reassembled_in,options_sack_dsack,resumed,alert_message,analysis_fast_retransmission
count,1983.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,9.0,9.000000,9.0,9.000000,0.0,0.0,0.0,0.0,0.0,0.0
mean,991.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,31.666667,100.0,31.666667,NaN,NaN,NaN,NaN,NaN,NaN
std,572.587111,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,11.067972,0.0,11.067972,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,20.000000,100.0,20.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,495.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,20.000000,100.0,20.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,991.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,41.000000,100.0,41.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,1486.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,41.000000,100.0,41.000000,NaN,NaN,NaN,NaN,NaN,NaN
max,1982.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.0,41.000000,100.0,41.000000,NaN,NaN,NaN,NaN,NaN,NaN
